In [1]:
# =============================================================================
# SETUP
# =============================================================================
import os
import sys
from dotenv import load_dotenv

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, os.path.join(project_root, 'src'))
    sys.path.insert(0, os.path.join(project_root, 'apps'))
    sys.path.insert(0, project_root)

env_path = os.path.join(project_root, '.env')
print(f"Loading environment variables from: {env_path}")
load_dotenv(env_path)
print(f"Tavily API Key loaded: {'TAVILY_API_KEY' in os.environ}")


Loading environment variables from: /home/juansebas7ian/deep-agents-from-scratch/.env
Tavily API Key loaded: True


In [2]:
# =============================================================================
# PASO 1: VALIDAR EL "MÚSCULO" (DynamoDB)
# =============================================================================
# Validamos src/neuro_agent/infrastructure/tools/dynamo_client.py

try:
    from neuro_agent.infrastructure.tools import save_task, get_context
    
    USER_TEST_ID = "user_val_test"
    TASK_DESC = "Unit Test Task: Validate DynamoDB Client"
    
    print("\n--- 1. Testing Write ---")
    # save_task is a @tool, so we invoke it or call it directly if possible.
    # It returns a string.
    try:
        if hasattr(save_task, 'invoke'):
             # If it's a StructuredTool
             write_result = save_task.invoke({'user_id': USER_TEST_ID, 'description': TASK_DESC})
        else:
             write_result = save_task(USER_TEST_ID, TASK_DESC)
    except Exception as ie:
        # Fallback if arguments are positional but tool expects kwarg dictionary for invoke
         write_result = save_task.invoke({'user_id': USER_TEST_ID, 'description': TASK_DESC})

    print(f"Write Result: {write_result}")
    
    print("\n--- 2. Testing Read ---")
    context = get_context.invoke({'user_id': USER_TEST_ID})
    print(f"Context Fetched: {context}")
    
    # Assert
    todos = context.get("todos", [])
    found = any(t.get('description') == TASK_DESC for t in todos)
    
    if found:
        print("\n✅ STEP 1 SUCCESS: Task written and retrieved from DynamoDB.")
    else:
        print("\n❌ STEP 1 FAILED: Task not found in fetched context.")
        
except ImportError as e:
    print(f"\n❌ IMPORT ERROR: {e}")
    print("Ensure src/neuro_agent/infrastructure/tools/dynamo_client.py exists and is in the python path.")
except Exception as e:
    print(f"\n❌ EXECUTION ERROR: {e}")



--- 1. Testing Write ---
Write Result: Tarea guardada correctamente (ID: 265963d2).

--- 2. Testing Read ---
Context Fetched: {'profile': {}, 'todos': [{'user_id': 'user_val_test', 'description': 'Unit Test Task: Validate DynamoDB Client', 'status': 'pending', 'task_id': '0a1ed834'}, {'user_id': 'user_val_test', 'description': 'Unit Test Task: Validate DynamoDB Client', 'status': 'pending', 'task_id': '0fa16192'}, {'user_id': 'user_val_test', 'description': 'Unit Test Task: Validate DynamoDB Client', 'status': 'pending', 'task_id': '1faab34d'}, {'user_id': 'user_val_test', 'description': 'Unit Test Task: Validate DynamoDB Client', 'status': 'pending', 'task_id': '265963d2'}, {'user_id': 'user_val_test', 'description': 'Unit Test Task: Validate DynamoDB Client', 'status': 'pending', 'task_id': '2ef8602f'}, {'user_id': 'user_val_test', 'description': 'Unit Test Task: Validate DynamoDB Client', 'status': 'pending', 'task_id': '319895e7'}, {'user_id': 'user_val_test', 'description': 'Unit

In [3]:
# =============================================================================
# PASO 2: VALIDAR EL "AGENTE IDIOTA" (Executor Handler)
# =============================================================================
import sys
import os

from apps.subagents.executor.handler import lambda_handler

print("\n--- Testing Lambda Handler ---")
USER_TEST_ID = "user_val_test"
INSTRUCTION = "Execute unit test task: Buy milk via Lambda"

mock_event = {
    "user_id": USER_TEST_ID,
    "explicit_instructions": INSTRUCTION
}

print(f"Invoking lambda_handler with event: {mock_event}")
# Invoke synchronously
response = lambda_handler(mock_event, None)
print(f"Response: {response}")

if response.get('statusCode') == 200:
    print("\n✅ STEP 2 SUCCESS: Executor Handler executed successfully and returned 200.")
else:
    print(f"\n❌ STEP 2 FAILED: Response code {response.get('statusCode')}")



--- Testing Lambda Handler ---
Invoking lambda_handler with event: {'user_id': 'user_val_test', 'explicit_instructions': 'Execute unit test task: Buy milk via Lambda'}
Response: {'statusCode': 200, 'body': '[{\'type\': \'text\', \'text\': \'<thinking>The user wants to execute a unit test task, specifically "Buy milk via Lambda". However, there are no tools available to execute tasks via Lambda or to run unit tests. The only available tool is to save a task in the database. Therefore, I will save this task as a reminder for the user to execute it later.</thinking>\\n\'}, {\'type\': \'tool_use\', \'name\': \'save_task\', \'input\': {\'user_id\': \'user_val_test\', \'description\': \'Buy milk via Lambda\'}, \'id\': \'tooluse_BAvtI32MxMFg7bByG0aX57\'}]'}

✅ STEP 2 SUCCESS: Executor Handler executed successfully and returned 200.


In [4]:
# =============================================================================
# PASO 3: VALIDAR HERRAMIENTAS INDIVIDUALES (Scrape)
# =============================================================================
from neuro_agent.infrastructure.tools import scrape_webpage

print("\n--- 3. Testing Scrape Tool ---")
URL = "https://example.com"
print(f"Scraping {URL}...")
try:
    # FIX: Use .invoke() for StructuredTools
    result = scrape_webpage.invoke({"url": URL})
    print(f"Result Snippet: {result[:200]}...")

    if "Example Domain" in result:
        print("\n✅ STEP 3 SUCCESS: Scrape tool fetched content correctly.")
    else:
        print("\n❌ STEP 3 FAILED: Content validation failed.")
except Exception as e:
    print(f"\n❌ STEP 3 ERROR: {e}")



--- 3. Testing Scrape Tool ---
Scraping https://example.com...
Result Snippet: Example Domain

Example Domain

This domain is for use in documentation examples without needing permission. Avoid use in operations.

[Learn more](https://iana.org/domains/example)...

✅ STEP 3 SUCCESS: Scrape tool fetched content correctly.


In [5]:
# =============================================================================
# PASO 4: VALIDAR EL "CEREBRO" (Supervisor + Tools Agnostiocas)
# =============================================================================
from langchain_aws import ChatBedrockConverse
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from neuro_agent.infrastructure.tools import (
    tavily_search, think_tool, ls, read_file, write_file, 
    write_todos, read_todos, get_today_str, scrape_webpage
)

print("\n--- 4. Testing Supervisor Agent Flow ---")

# 1. Setup Model & Tools
llm = ChatBedrockConverse(model="us.amazon.nova-pro-v1:0", region_name="us-east-1", temperature=0.0)
tools = [ls, read_file, write_file, write_todos, read_todos, think_tool, tavily_search, scrape_webpage]

# 2. Create Agent
agent = create_agent(llm, tools=tools)

# 3. Run Query
USER_QUERY = "Give me a brief overview of Model Context Protocol (MCP) using a web search. Read the docs if possible."
print(f"Invoking Agent with query: '{USER_QUERY}'")
print("Observing tool calls... (Expecting: tavily_search -> scrape_webpage)")

try:
    stream = agent.stream(
        {"messages": [HumanMessage(content=USER_QUERY)]},
        stream_mode="values"
    )

    tool_calls_observed = set()

    for event in stream:
        if "messages" in event:
            last_msg = event["messages"][-1]
            
            # Check for Tool Calls
            tool_calls = getattr(last_msg, 'tool_calls', [])
            if tool_calls:
                for tc in tool_calls:
                    tool_name = tc['name']
                    print(f"🔧 Tool Call: {tool_name}")
                    tool_calls_observed.add(tool_name)
            
            # Check for Content
            if hasattr(last_msg, 'content') and last_msg.content and not tool_calls:
                 if last_msg.type == 'ai':
                    print(f"🤖 Answer Snippet: {last_msg.content[:100]}...")

    # 4. Assertions
    print("\n--- Validation Report ---")
    if 'tavily_search' in tool_calls_observed and 'scrape_webpage' in tool_calls_observed:
        print("✅ STEP 4 SUCCESS: Agent used Search AND Scrape!")
    elif 'tavily_search' in tool_calls_observed:
        print("⚠️ STEP 4 WARNING: Agent searched but did NOT scrape.")
    else:
        print(f"❌ STEP 4 FAILURE: Missing critical tools. Observed: {tool_calls_observed}")

except Exception as e:
    print(f"\n❌ STEP 4 ERROR: {e}")



--- 4. Testing Supervisor Agent Flow ---
Invoking Agent with query: 'Give me a brief overview of Model Context Protocol (MCP) using a web search. Read the docs if possible.'
Observing tool calls... (Expecting: tavily_search -> scrape_webpage)
🔧 Tool Call: tavily_search
🔧 Tool Call: read_file
🤖 Answer Snippet: <thinking> It seems that the file "model_context_protocol_summary_A4WO2USJ.md" could not be found. I...

--- Validation Report ---
⚠️ STEP 4 WARNING: Agent searched but did NOT scrape.
